In [ ]:
!pip install agentpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.9/778.9 kB 24.9 MB/s eta 0:00:00


# Descripción del sistema multiagente
Esta simulación urbana en AgentPy modela un cruce de intersección (crossroad) donde conviven peatones y vehículos en una cuadrícula. Se definen las aceras, carreteras y múltiples pasos peatonales, con obstáculos aleatorios en banquetas. Los peatones usan el algoritmo Weighted A* para planear rutas desde un origen aleatorio hasta un destino común, evitando obstáculos y el tráfico. Los vehículos circulan por las calles de forma lineal, reapareciendo al salir del área hasta cumplir con un número definido.

# Que queda pendiente?


*   **Destino de los peatones** : Por ahora todos los agentes peatones se dirigen al mismo lugar para facilitar la implementación, pero para la versión final, lo ideal sería que los agentes tengan diferentes destinos.
*   **Implementación del "Agente Obstaculo"** : Por ahora los osbtaculos simplemente vendrian a ser los "comercios informales", pero debemos implementar un nuevo agente que sera un peatón "lento", en este caso representando a una posible persona en silla de ruedas o adultos mayores.
*   **Implementación del "Agente Autoridad"** : El agente autoridad, simplemente se dedicara a remover ciertos obstaculos como los comercios informales y retornara a un mismo punto una vez lo haya logrado.
*   **Implementación de un sistema de semaforos** : Para alcanzar un nivel de realismo mas acertado, se buscara la implementación de un sistema de semaforos que indicara a los peatones cuando cruzar y cuando no hacerlo, adicionalmente, se incrementara la cantidad de vehiculos circulando que también reaccionaran ante el semaforo.

* **Peatones arriesgados** : La cantidad de obstaculos en las banquetas provocara que ciertos agentes tomen decisiones mas arriesgadas como bajar al arroyo.





In [ ]:
# Avance 3 - Simulación 70% completa

import agentpy as ap
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML

# 2. Definir la función Weighted A* (idéntica a su versión original)
def weighted_a_star(grid, start, goal, weight=1):
    # Funciones auxiliares
    def heuristic(a, b):
        return abs(a[0] - b[0]) + abs(a[1] - b[1])

    def neighbors(pos):
        # Vecinos en Von Neumann (arriba, abajo, izquierda, derecha)
        for d in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            nr, nc = pos[0] + d[0], pos[1] + d[1]
            if 0 <= nr < grid.shape[0] and 0 <= nc < grid.shape[1]:
                yield (nr, nc)

    open_set = {start}
    came_from = {}
    g_score = {start: 0}
    f_score = {start: weight * heuristic(start, goal)}

    while open_set:
        current = min(open_set, key=lambda pos: f_score.get(pos, np.inf))
        if current == goal:
            # Reconstruir camino
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            return path[::-1]  # Retorna el camino invertido

        open_set.remove(current)
        for neighbor in neighbors(current):
            if grid[neighbor] == 1 or grid[neighbor] == 2:
                continue  # Omitir celdas de obstáculo y carretera
            tentative_g = g_score[current] + 1
            if tentative_g < g_score.get(neighbor, np.inf):
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g
                f_score[neighbor] = tentative_g + weight * heuristic(neighbor, goal)
                open_set.add(neighbor)

    return None  # No se encontró camino

# 3. Definir el modelo CrossroadModel
class CrossroadModel(ap.Model):
    def setup(self):
        # Parámetros recibidos
        self.width = self.p.width
        self.height = self.p.height
        self.obs_density = self.p.obs_density
        self.weight = self.p.weight
        self.max_pedestrians = self.p.max_pedestrians
        self.max_vehicles = self.p.max_vehicles

        # Crear grid vacío para pathfinding y visualización
        # 0: banqueta / cruce, 1: obstáculo en banqueta, 2: carretera, 3: cruce peatonal
        self.grid = np.zeros((self.height, self.width), dtype=int)

        # Definir calles (horizontal y vertical)
        # Calle horizontal (dos filas centrales)
        self.horizontal_road_rows = [self.height // 2 - 1, self.height // 2]
        # Calle vertical (dos columnas centrales)
        self.vertical_road_cols = [self.width // 2 - 1, self.width // 2]

        # Marcar celdas de carretera horizontal
        for r in self.horizontal_road_rows:
            self.grid[r, :] = 2

        # Marcar celdas de carretera vertical
        for c in self.vertical_road_cols:
            self.grid[:, c] = 2

        # Crear cruces peatonales que conecten TODAS las veredas

        # Cruces peatonales HORIZONTALES (para cruzar la calle horizontal de norte a sur)
        # Los colocamos en varias columnas, no solo en el centro
        crosswalk_horizontal_cols = [
            self.width // 4,           # 1/4 del ancho
            self.width // 2,           # Centro (ya existía)
            3 * self.width // 4        # 3/4 del ancho
        ]

        for col in crosswalk_horizontal_cols:
            if col < self.width:  # Verificar que esté dentro del mapa
                for row in self.horizontal_road_rows:
                    self.grid[row, col] = 3  # Cruce peatonal

        # Cruces peatonales VERTICALES (para cruzar la calle vertical de oeste a este)
        crosswalk_vertical_rows = [
            self.height // 4,          # 1/4 de la altura
            self.height // 2,          # Centro (ya existía)
            3 * self.height // 4       # 3/4 de la altura
        ]

        for row in crosswalk_vertical_rows:
            if row < self.height:  # Verificar que esté dentro del mapa
                for col in self.vertical_road_cols:
                    self.grid[row, col] = 3  # Cruce peatonal

        # Guardar estas variables para uso posterior
        self.crosswalk_horizontal_cols = crosswalk_horizontal_cols
        self.crosswalk_vertical_rows = crosswalk_vertical_rows

        # Definir áreas de banqueta (4 cuadrantes)
        self.sidewalk_north = [(r, c) for r in range(0, self.horizontal_road_rows[0])
                              for c in range(self.width)
                              if c not in self.vertical_road_cols]

        self.sidewalk_south = [(r, c) for r in range(self.horizontal_road_rows[1] + 1, self.height)
                              for c in range(self.width)
                              if c not in self.vertical_road_cols]

        self.sidewalk_west = [(r, c) for r in range(self.height)
                             for c in range(0, self.vertical_road_cols[0])
                             if r not in self.horizontal_road_rows]

        self.sidewalk_east = [(r, c) for r in range(self.height)
                             for c in range(self.vertical_road_cols[1] + 1, self.width)
                             if r not in self.horizontal_road_rows]

        # Colocar obstáculos en banquetas aleatoriamente
        all_sidewalks = self.sidewalk_north + self.sidewalk_south + self.sidewalk_west + self.sidewalk_east

        # Definir accesos a TODOS los cruces peatonales (no colocar obstáculos aquí)
        crosswalk_access = []

        # Accesos a cruces horizontales
        for col in crosswalk_horizontal_cols:
            if col < self.width:
                crosswalk_access.extend([
                    (self.horizontal_road_rows[0] - 1, col),  # Acceso desde el norte
                    (self.horizontal_road_rows[1] + 1, col),  # Acceso desde el sur
                ])

        # Accesos a cruces verticales
        for row in crosswalk_vertical_rows:
            if row < self.height:
                crosswalk_access.extend([
                    (row, self.vertical_road_cols[0] - 1),    # Acceso desde el oeste
                    (row, self.vertical_road_cols[1] + 1),    # Acceso desde el este
                ])

        for r, c in all_sidewalks:
            if np.random.rand() < self.obs_density and (r, c) not in crosswalk_access:
                self.grid[r, c] = 1

        # DESTINO FIJO: Punto específico donde todos los peatones deben llegar
        # Lo colocamos en una esquina para forzar cruces largos
        self.destination_point = (self.height - 3, self.width - 3)  # Esquina sureste
        # Asegurar que el destino esté libre
        self.grid[self.destination_point] = 0

        # Crear listas de agentes
        self.pedestrians = ap.AgentList(self, self.max_pedestrians, Pedestrian)
        self.vehicles = ap.AgentList(self, self.max_vehicles, Vehicle)

        # Variables para contar cuántos completan recorrido
        self.completed_pedestrians = 0
        self.completed_vehicles = 0

        # Spawn inicial de agentes
        for ped in self.pedestrians:
            ped.origin = self.sample_pedestrian_origin()
            ped.destination = self.destination_point  # TODOS van al mismo destino
            ped.setup_agent()
        for veh in self.vehicles:
            veh.setup_agent()

    def sample_pedestrian_origin(self):
        # Obtener todas las posiciones de banqueta disponibles
        all_sidewalk_positions = []

        # Agregar todas las banquetas
        for r, c in (self.sidewalk_north + self.sidewalk_south +
                     self.sidewalk_west + self.sidewalk_east):
            if self.grid[r, c] == 0:  # Solo posiciones transitables
                all_sidewalk_positions.append((r, c))

        if not all_sidewalk_positions:
            # Fallback: usar una posición segura
            return (2, 2)

        # Elegir una posición aleatoria
        return self.random.choice(all_sidewalk_positions)

    def step(self):
        # Hacer step de vehículos primero (se mueven en línea recta)
        self.vehicles.step()
        # Luego hacer step de peatones
        self.pedestrians.step()

    def end(self):
        # Detener simulación cuando ambos hayan completado sus metas
        if (self.completed_pedestrians >= self.max_pedestrians) and (self.completed_vehicles >= self.max_vehicles):
            self.stop()

# 4. Definir clase Pedestrian (peatones)
class Pedestrian(ap.Agent):
    def setup_agent(self):
        # Colocar agente en su origen
        self.pos = self.origin

        # Calcular ruta usando Weighted A*
        self.path = weighted_a_star(self.model.grid, self.origin, self.destination, weight=self.model.weight)

        if not self.path:
            # Si no se encuentra ruta directa, usar ruta simple
            self.path = [self.origin, self.destination]

        self.step_index = 0
        self.completed = False

        # Debug: imprimir información del peatón
        print(f"Peatón creado: {self.origin} → {self.destination}, ruta de {len(self.path)} pasos")

    def step(self):
        if self.completed:
            return

        if self.step_index < len(self.path):
            self.pos = self.path[self.step_index]
            self.step_index += 1

            # Debug: mostrar progreso cada 20 pasos
            if self.step_index % 20 == 0:
                print(f"Peatón en paso {self.step_index}/{len(self.path)}, posición: {self.pos}")
        else:
            # Llegó a destino
            print(f"✓ Peatón completó recorrido: {self.origin} → {self.destination} en {len(self.path)} pasos")
            self.completed = True
            self.model.completed_pedestrians += 1

            # Si aún no han completado todos, respawnear nuevo peatón
            if self.model.completed_pedestrians < self.model.max_pedestrians:
                new_origin = self.model.sample_pedestrian_origin()
                self.origin = new_origin
                self.destination = self.model.destination_point
                self.setup_agent()
            else:
                # Si ya terminaron, eliminar agente de la lista
                if self in self.model.pedestrians:
                    self.model.pedestrians.remove(self)

# 5. Definir clase Vehicle (vehículos en línea recta)
class Vehicle(ap.Agent):
    def setup_agent(self):
        # Elegir aleatoriamente una de las 4 direcciones posibles
        direction_choice = self.model.random.randint(0, 3)

        if direction_choice == 0:
            # Carretera horizontal superior: izquierda → derecha
            self.row = self.model.horizontal_road_rows[0]
            self.col = 0
            self.direction = (0, 1)  # (delta_row, delta_col)
            self.is_horizontal = True
        elif direction_choice == 1:
            # Carretera horizontal inferior: derecha → izquierda
            self.row = self.model.horizontal_road_rows[1]
            self.col = self.model.width - 1
            self.direction = (0, -1)
            self.is_horizontal = True
        elif direction_choice == 2:
            # Carretera vertical izquierda: arriba → abajo
            self.row = 0
            self.col = self.model.vertical_road_cols[0]
            self.direction = (1, 0)
            self.is_horizontal = False
        else:
            # Carretera vertical derecha: abajo → arriba
            self.row = self.model.height - 1
            self.col = self.model.vertical_road_cols[1]
            self.direction = (-1, 0)
            self.is_horizontal = False

        self.pos = (self.row, self.col)
        self.completed = False

    def step(self):
        if self.completed:
            return

        new_row = self.row + self.direction[0]
        new_col = self.col + self.direction[1]

        # Verificar si salió del mapa
        if (new_row < 0 or new_row >= self.model.height or
            new_col < 0 or new_col >= self.model.width):
            # Llegó al extremo
            self.completed = True
            self.model.completed_vehicles += 1
            # Si no han completado el número total, respawnear
            if self.model.completed_vehicles < self.model.max_vehicles:
                self.setup_agent()
            else:
                # Si ya terminaron, eliminar agente de la lista
                if self in self.model.vehicles:
                    self.model.vehicles.remove(self)
        else:
            # Avanzar
            self.row = new_row
            self.col = new_col
            self.pos = (self.row, self.col)

# 6. Función de visualización
def plot(model, ax):
    ax.clear()
    # Mapa de colores para cada tipo de celda
    cmap = {
        0: 'lightgray',  # Banqueta
        1: 'sienna',     # Obstáculo informal en banqueta
        2: 'gray',       # Carretera
        3: 'white'       # Cruce peatonal
    }
    # Dibujar cada celda del grid como un rectángulo
    for r in range(model.height):
        for c in range(model.width):
            ax.add_patch(plt.Rectangle(
                (c, model.height - 1 - r), 1, 1,
                color=cmap[model.grid[r, c]], ec='black', lw=0.1
            ))

    #Dibujar el destino dorado
    dest_r, dest_c = model.destination_point
    ax.add_patch(plt.Rectangle(
        (dest_c, model.height - 1 - dest_r), 1, 1,
        color='gold', ec='black', lw=2
    ))

    # Agregar estrella dorada en el centro del destino para mayor visibilidad
    ax.scatter(dest_c + 0.5, model.height - 1 - dest_r + 0.5,
               c='gold', s=200, marker='*', edgecolors='black', linewidth=1,
               label='Destino', zorder=10)

    # Dibujar peatones vivos
    ped_x = [p.pos[1] + 0.5 for p in model.pedestrians if not p.completed]
    ped_y = [model.height - 1 - p.pos[0] + 0.5 for p in model.pedestrians if not p.completed]
    if ped_x:  # Solo dibujar si hay peatones
        ax.scatter(ped_x, ped_y, c='blue', s=50, label='Peatón')

    # Dibujar vehículos vivos
    veh_x = [v.pos[1] + 0.5 for v in model.vehicles if not v.completed]
    veh_y = [model.height - 1 - v.pos[0] + 0.5 for v in model.vehicles if not v.completed]
    if veh_x:  # Solo dibujar si hay vehículos
        ax.scatter(veh_x, veh_y, c='red', s=50, marker='s', label='Vehículo')

    ax.set_xlim(0, model.width)
    ax.set_ylim(0, model.height)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(f"Crossroad Simulation - Paso: {model.t}")
    ax.legend(loc='upper right')

# 7. Parámetros para instanciar el modelo
params = {
    'width': 25,
    'height': 25,
    'obs_density': 0.05,     # Reducir obstáculos para facilitar pathfinding
    'weight': 3,             # Reducir peso para rutas más directas
    'max_pedestrians': 5,    # Menos peatones para debugging
    'max_vehicles': 8,       # Menos vehículos
    'steps': 400,            # Más pasos para rutas más largas
}

# 8. Crear instancia del modelo
model = CrossroadModel(params)

# 9. Crear figura y ejes para la animaciónS
fig, ax = plt.subplots(figsize=(8, 6))

# 10. Generar la animación usando ap.animate
animation = ap.animate(
    model,        # instancia de CrossroadModel
    fig,          # figura de matplotlib
    ax,           # ejes en los que se dibuja
    plot,         # función de visualización (model, ax) → dibuja el estado actual
    steps=60,    # número máximo de pasos
    seed=42       # semilla para reproducibilidad (opcional)
)

# 11. Mostrar la animación en Colab o Jupyter
HTML(animation.to_jshtml())

Peatón creado: (15, 14) → (22, 22), ruta de 16 pasos
Peatón creado: (13, 23) → (22, 22), ruta de 11 pasos
Peatón creado: (10, 23) → (22, 22), ruta de 22 pasos
Peatón creado: (6, 20) → (22, 22), ruta de 23 pasos
Peatón creado: (4, 17) → (22, 22), ruta de 28 pasos
✓ Peatón completó recorrido: (13, 23) → (22, 22) en 11 pasos
Peatón creado: (3, 7) → (22, 22), ruta de 35 pasos
✓ Peatón completó recorrido: (15, 14) → (22, 22) en 16 pasos
Peatón creado: (15, 10) → (22, 22), ruta de 34 pasos
Peatón en paso 20/22, posición: (21, 21)
Peatón en paso 20/23, posición: (20, 21)
Peatón en paso 20/28, posición: (16, 20)
✓ Peatón completó recorrido: (10, 23) → (22, 22) en 22 pasos
Peatón creado: (1, 8) → (22, 22), ruta de 36 pasos
✓ Peatón completó recorrido: (6, 20) → (22, 22) en 23 pasos
Peatón creado: (9, 1) → (22, 22), ruta de 39 pasos
✓ Peatón completó recorrido: (4, 17) → (22, 22) en 28 pasos
Peatón en paso 20/35, posición: (11, 18)
Peatón en paso 20/34, posición: (18, 12)
Peatón en paso 20/36, p